In [1]:
import sys
from pathlib import Path
import pandas as pd

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils
from config import date_str, DOWNLOAD_DIR, DEST_DIR

path_data = f'..\data\{date_str}_df_finviz_merged.parquet'
# path_corr = '..\data\df_corr_emv_matrix.parquet'
# path_cov = '..\data\df_cov_emv_matrix.parquet'
# path_output = f'..\picks\{date_str}_portf.txt'


print(utils.__file__)  # Should point to your src/utils.py
print(f'path_date: {path_data}')
# print(f'path_corr: {path_corr}')
# print(f'path_cov: {path_cov}')  
# print(f'path_output: {path_output}')
# print((f'date_str: {date_str}'))



Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']
c:\Users\ping\Files_win10\python\py311\stocks\src\utils.py
path_date: ..\data\2025-04-24_df_finviz_merged.parquet


In [2]:
import pandas as pd
# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell
pd.set_option('display.max_rows', 100)
# pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.4f}'.format)

In [3]:
import pandas as pd

def get_column_values_above_threshold(df, column_name='Avg Volume, M', threshold=0.75):
  """
  Analyzes the number and percentage of values in a DataFrame column that are above a specified threshold,
  and returns the filtered DataFrame.

  Args:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to analyze. Defaults to 'Avg Volume, M'.
    threshold (float): The threshold value to compare against. Defaults to 1.00.

  Returns:
    pd.DataFrame: A DataFrame containing only the rows where the specified column's value is above the threshold.
  """
  
  count_before = len(df)
  above_threshold_df = df[df[column_name] > threshold]
  count_after = len(above_threshold_df)
  percentage = (count_after / len(df)) * 100

  print(f"count_before: {count_before}")
  print(f"count_after above threshold ({threshold}): {count_after}")
  print(f"Percentage above threshold ({threshold}): {percentage:.2f}%")

  return above_threshold_df


In [4]:
df_data = pd.read_parquet(path_data)

# liquidity filter, Avg Volume, M > 0.75M
df_data = get_column_values_above_threshold(df_data, column_name='Avg Volume, M', threshold=0.75)

# Drop specified columns with NaNs in df_data
df_data = df_data.drop(['All-Time High %', 'All-Time Low %', 'Dividend %'], axis=1)

# df_corr = pd.read_parquet(path_corr)
# df_cov = pd.read_parquet(path_cov)

# print(f'\ndf_cov.shape: {df_cov.shape}')
# display(df_cov.head())

# print(f'\ndf_corr.shape: {df_corr.shape}')
# display(df_corr.head())

print(f'\ndf_data.shape: {df_data.shape}')
display(df_data.head())
display((df_data.describe()))

count_before: 1136
count_after above threshold (0.75): 927
Percentage above threshold (0.75): 81.60%

df_data.shape: (927, 59)


,Company,Info,"MktCap, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Fwd P/E,P/S,P/B,P/FCF,ROE %,Debt/Eq,Oper M %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",3130150.0000,1.2800,51.3800,-16.7900,7.8743,7.2600,-6.8700,-6.8400,-11.8900,25.6500,3.0100,-5.0800,-8.5000,-16.6500,23.1400,-19.8900,25.3700,9.7000,4.6552,2.5400,4.9500,47.0682,61.2700,0.7700,208.3700,1.8400,26.1900,7.9100,46.9500,31.8400,136.5200,1.4500,31.7600,80.6701,9957.0310,888.0428,9.6300,23.0568,3.9049,6.2602,11.1003,2.5127,0.8612,1.5845,1.1613,0.1840,0.2850,1.0358,-0.9594,-1.3857,0.8283,-0.4254,-0.6087,0.9178,0.6840,1.0178,1.1436
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2879180.0000,0.9800,53.8800,-8.1100,7.8470,4.2200,-1.9900,-13.3000,-7.5200,-3.4100,3.0700,-0.0700,-6.8300,-7.6300,12.3300,-17.3100,12.3300,12.9600,3.3462,2.4400,3.5100,22.1123,25.2500,0.8800,387.3000,3.4500,26.0500,11.0000,9.5100,41.1100,34.2900,0.3400,44.9600,44.4284,9957.0310,888.0428,8.2031,17.5381,3.2096,1.2219,1.8995,1.2080,1.3725,2.8900,1.2826,0.5617,1.0440,1.1112,-1.5374,-2.2744,0.7455,-0.7315,-1.0429,0.8716,-0.1273,-0.1783,0.9771
NVDA,NVIDIA Corp,"Technology, Semiconductors",2596890.0000,2.1100,48.2400,-20.7500,9.8235,1.8600,-11.8200,-27.7100,-25.9400,33.8400,1.0400,-7.9700,-15.3100,-25.8000,22.8700,-30.5000,36.0600,6.8700,6.4549,3.5400,6.4400,217.5547,315.2700,0.6900,106.4300,3.6200,18.8900,19.9000,32.8400,42.6700,119.1800,0.1300,62.4200,347.0916,9957.0310,888.0428,5.0026,8.6786,2.0934,-0.2734,-0.3631,0.9587,1.1871,2.2707,1.2373,-0.5525,-0.9010,0.9046,-0.6254,-0.9094,0.8976,-0.6372,-0.8691,0.8943,0.6602,0.9563,1.1190
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",1979630.0000,1.3000,50.4600,-14.9700,11.4870,7.0000,-9.3200,-20.7600,-1.3400,5.2500,2.4600,-5.4800,-6.3900,-20.0900,15.5900,-23.0800,23.0400,9.4800,5.0820,3.4200,5.2500,41.9393,49.4300,0.8500,186.5400,3.2900,24.9900,3.1000,6.9100,60.2100,24.2900,0.5200,10.8700,85.0378,9957.0310,888.0428,9.2623,20.4775,3.5799,1.8775,3.3802,1.3110,1.3845,2.7160,1.2522,-0.3348,-0.5044,0.9443,-2.0411,-2.8035,0.7029,-0.0803,-0.1169,0.9864,0.2643,0.3748,1.0472
GOOGL,Alphabet Inc,"Communication Services, Internet Content & Inf...",1954980.0000,1.0000,51.7200,-15.8600,7.8621,3.8800,-6.6100,-19.5500,-2.9200,1.9200,3.5800,-3.2500,-7.6200,-14.8000,13.3400,-23.0700,13.3400,5.8600,3.6791,2.6800,3.8400,42.6839,35.0800,1.2200,159.2800,2.5300,15.9000,5.5900,5.9800,26.8700,32.9100,0.0900,32.4600,1786.4239,9957.0310,888.0428,8.6365,18.0698,3.2766,3.2829,5.9107,1.5669,1.9599,4.0045,1.3816,-0.2670,-0.4194,0.9573,-2.0078,-2.6746,0.7181,-0.4415,-0.6298,0.9290,0.1135,0.1640,1.0201


,"MktCap, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Fwd P/E,P/S,P/B,P/FCF,ROE %,Debt/Eq,Oper M %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,926.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,898.0000,921.0000,887.0000,784.0000,886.0000,884.0000,918.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000
mean,68185.1564,1.0574,52.2084,-2.2813,6.2209,4.9248,-4.2898,-6.9039,-2.8921,15.9483,2.8571,-2.0177,-2.9515,-14.8572,18.3423,-22.3450,38.9037,4.6340,4.3776,3.3575,4.6446,5.7708,6.4600,0.9169,107.9090,2.0581,20.1650,8.7513,8.3489,37.2420,32.5383,1.7576,2.5578,164.9045,6630.4213,591.7126,6.5286,269.8387,34.9744,5.4370,32.4176,6.3460,0.5957,1.5130,1.1927,0.3073,0.6756,1.0966,-0.3694,-0.3728,0.9647,-0.0592,0.0149,1.0044,0.3218,0.5511,1.0743
std,220474.9086,0.5321,6.7207,17.8295,5.3104,5.4265,8.4611,16.3314,22.2828,65.2262,5.0994,7.3581,15.2153,10.0759,10.6075,14.2289,76.3480,5.5506,1.5078,1.2108,1.6650,12.4519,15.2181,0.5095,123.3082,2.5654,17.0861,78.0859,28.9454,90.4793,375.1463,5.5948,254.0630,1525.2870,4676.1402,416.5443,9.1007,921.7743,116.1359,4.1880,256.4161,48.4530,2.0593,3.6547,0.4384,1.5377,2.4506,0.3201,1.4519,2.0934,0.2713,1.0045,1.4694,0.1890,0.8210,1.2703,0.1688
min,5480.0000,-1.1600,28.3900,-54.6800,-14.7306,-27.4800,-28.2600,-55.2900,-54.9400,-85.5900,-18.6600,-24.3100,-49.8400,-61.4400,-5.2300,-95.6000,-2.1800,0.0700,0.3745,0.2700,0.3900,0.2778,0.7532,0.2300,2.3800,-18.5200,3.3000,0.0600,0.3200,0.3900,-4584.4700,0.0000,-5494.8600,-4764.4477,-15.8745,0.0000,-28.9318,-14.3381,0.0000,-8.8184,-8.3690,0.0512,-7.8879,-7.5704,0.2411,-5.5193,-5.7540,0.3772,-4.2915,-4.9259,0.4728,-2.9721,-3.4865,0.5795,-1.9473,-2.4592,0.6913
25%,10195.0000,0.7200,48.1950,-13.4850,2.9538,1.8150,-9.5550,-17.6650,-16.8950,-12.7150,0.1250,-6.4500,-12.1950,-20.6700,11.7150,-31.1850,15.6100,1.4350,3.2253,2.5150,3.4200,1.2878,1.5500,0.6500,35.2650,0.5750,10.6000,1.3200,1.7300,12.2675,7.1750,0.3575,9.1575,6.1123,26.8438,3.3914,1.6825,3.0131,1.3347,2.9651,4.9372,1.6001,-0.7575,-1.0393,0.8769,-0.6531,-0.9102,0.8879,-1.3799,-1.8379,0.7751,-0.7359,-1.0153,0.8695,-0.2444,-0.3300,0.9539
50%,20340.0000,1.0200,51.8000,-3.7600,5.9524,4.6900,-4.7200,-8.1100,-5.8100,7.9800,2.3100,-2.2800,-4.2400,-13.5800,16.3600,-20.3300,24.0700,2.9300,4.1134,3.1300,4.3200,2.2595,2.7300,0.8100,70.2900,1.8600,15.8100,2.6400,3.0100,20.3800,13.2750,0.7450,16.6100,23.9880,9957.0310,888.0428,6.2175,13.3910,2.5893,5.2635,10.4202,2.3230,0.6782,1.1076,1.1290,0.1905,0.3104,1.0357,-0.5367,-0.7692,0.9071,-0.1354,-0.1957,0.9756,0.3381,0.4813,1.0643
75%,53030.0000,1.3000,56.2200,7.8700,8.8394,7.4600,0.4150,3.1650,7.8650,26.7400,5.3850,2.0100,5.9950,-7.1250,22.3000,-11.5700,40.1600,5.7750,5.2814,4.0450,5.5000,5.2748,5.6850,1.0350,133.1100,3.3500,22.9925,5.5600,6.3650,32.7325,24.3975,1.5300,25.1675,62.0671,9957.0310,888.0428,9.5964,29.4031,4.6436,7.5977,18.5123,3.5185,1.9980,3.6136,1.4306,1.1906,1.7973,1.2357,0.5086,0.7365,1.0930,0.6187,0.9130,1.1201,0.8627,1.2875,1.1703
max,3130150.0000,5.0600,78.9900,105.6600,45.5194,59.2200,81.3200,52.8700,152.4100,1014.3500,62.3100,75.0500,98.7000,8.4200,136.0100,8.2900,1647.6200,47.5700,11.4072,10.8800,12.2300,217.5547,315.2700,9.2800,1096.8700,15.4900,166.1200,1723.5200,4

In [5]:
import pandas as pd
import os
import datetime

# --- Configuration ---
STATS_HISTORY_FILE = 'market_daily_summary_stats_history.csv' # Where to store the history

def append_daily_stats(df_daily_data, date_str, filename=STATS_HISTORY_FILE):
    """
    Calculates descriptive stats for df_daily_data, flattens them,
    and appends/updates them as a row in a historical CSV file.

    Args:
        df_daily_data (pd.DataFrame): DataFrame containing the raw data for ALL stocks for ONE specific day.
        date_str (str): The date for this data in 'YYYY-MM-DD' format.
        filename (str): The path to the CSV file storing the historical summary stats.
    """
    print(f"Processing stats for date: {date_str}")

    # 1. Calculate descriptive statistics for the input day's data
    try:
        daily_stats_raw = df_daily_data.describe()
        print("Successfully calculated describe() stats.")
    except Exception as e:
        print(f"Error calculating describe() stats: {e}")
        return # Stop if calculation fails

    # 2. Flatten the describe() output into a single row DataFrame
    try:
        # Unstack turns the stats (mean, std, etc.) into columns alongside the original metrics
        stats_flat = daily_stats_raw.unstack().to_frame().T
        # Create clearer column names like 'Metric_Statistic' (e.g., 'RSI_mean', 'SMA50 %_50%')
        stats_flat.columns = ['_'.join(col).strip() for col in stats_flat.columns.values]
        # Add the date to this row
        stats_flat['Date'] = pd.to_datetime(date_str)
        # Set the Date as the index for this single row
        stats_flat = stats_flat.set_index('Date')
        print("Successfully flattened stats into a single row.")
        # print(stats_flat) # Optional: print the flattened stats for verification

    except Exception as e:
        print(f"Error flattening the stats DataFrame: {e}")
        return # Stop if flattening fails

    # 3. Load existing history, append/update, and save
    try:
        df_summary_history = pd.DataFrame() # Initialize empty
        if os.path.exists(filename):
            print(f"Loading existing history from: {filename}")
            df_summary_history = pd.read_csv(filename, index_col='Date', parse_dates=True)

        # Check if the date already exists - update if it does, append if it doesn't
        current_date = pd.to_datetime(date_str)
        if current_date in df_summary_history.index:
            print(f"Updating existing entry for {date_str}")
            # Update existing row(s) - combines columns, replacing old values for this date
            df_summary_history.update(stats_flat)
            # Alternative: df_summary_history.loc[current_date] = stats_flat.iloc[0] # Direct overwrite
        else:
            print(f"Appending new entry for {date_str}")
            df_summary_history = pd.concat([df_summary_history, stats_flat])

        # Ensure the history is sorted by date
        df_summary_history = df_summary_history.sort_index()

        # Save the updated history back to CSV
        df_summary_history.to_csv(filename, index=True) # index=True saves the Date index
        print(f"Successfully saved updated stats history to: {filename}")

    except Exception as e:
        print(f"Error loading/saving the stats history file '{filename}': {e}")



In [6]:
# Simulate processing for a couple of days
print("-" * 20, "Day 1", "-" * 20)
append_daily_stats(df_data, date_str)

-------------------- Day 1 --------------------
Processing stats for date: 2025-04-24
Successfully calculated describe() stats.
Successfully flattened stats into a single row.
Loading existing history from: market_daily_summary_stats_history.csv
Updating existing entry for 2025-04-24
Successfully saved updated stats history to: market_daily_summary_stats_history.csv


In [7]:
print(f'date_str: {date_str}')
df_data.describe()

date_str: 2025-04-24


,"MktCap, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Fwd P/E,P/S,P/B,P/FCF,ROE %,Debt/Eq,Oper M %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,926.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,898.0000,921.0000,887.0000,784.0000,886.0000,884.0000,918.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000,927.0000
mean,68185.1564,1.0574,52.2084,-2.2813,6.2209,4.9248,-4.2898,-6.9039,-2.8921,15.9483,2.8571,-2.0177,-2.9515,-14.8572,18.3423,-22.3450,38.9037,4.6340,4.3776,3.3575,4.6446,5.7708,6.4600,0.9169,107.9090,2.0581,20.1650,8.7513,8.3489,37.2420,32.5383,1.7576,2.5578,164.9045,6630.4213,591.7126,6.5286,269.8387,34.9744,5.4370,32.4176,6.3460,0.5957,1.5130,1.1927,0.3073,0.6756,1.0966,-0.3694,-0.3728,0.9647,-0.0592,0.0149,1.0044,0.3218,0.5511,1.0743
std,220474.9086,0.5321,6.7207,17.8295,5.3104,5.4265,8.4611,16.3314,22.2828,65.2262,5.0994,7.3581,15.2153,10.0759,10.6075,14.2289,76.3480,5.5506,1.5078,1.2108,1.6650,12.4519,15.2181,0.5095,123.3082,2.5654,17.0861,78.0859,28.9454,90.4793,375.1463,5.5948,254.0630,1525.2870,4676.1402,416.5443,9.1007,921.7743,116.1359,4.1880,256.4161,48.4530,2.0593,3.6547,0.4384,1.5377,2.4506,0.3201,1.4519,2.0934,0.2713,1.0045,1.4694,0.1890,0.8210,1.2703,0.1688
min,5480.0000,-1.1600,28.3900,-54.6800,-14.7306,-27.4800,-28.2600,-55.2900,-54.9400,-85.5900,-18.6600,-24.3100,-49.8400,-61.4400,-5.2300,-95.6000,-2.1800,0.0700,0.3745,0.2700,0.3900,0.2778,0.7532,0.2300,2.3800,-18.5200,3.3000,0.0600,0.3200,0.3900,-4584.4700,0.0000,-5494.8600,-4764.4477,-15.8745,0.0000,-28.9318,-14.3381,0.0000,-8.8184,-8.3690,0.0512,-7.8879,-7.5704,0.2411,-5.5193,-5.7540,0.3772,-4.2915,-4.9259,0.4728,-2.9721,-3.4865,0.5795,-1.9473,-2.4592,0.6913
25%,10195.0000,0.7200,48.1950,-13.4850,2.9538,1.8150,-9.5550,-17.6650,-16.8950,-12.7150,0.1250,-6.4500,-12.1950,-20.6700,11.7150,-31.1850,15.6100,1.4350,3.2253,2.5150,3.4200,1.2878,1.5500,0.6500,35.2650,0.5750,10.6000,1.3200,1.7300,12.2675,7.1750,0.3575,9.1575,6.1123,26.8438,3.3914,1.6825,3.0131,1.3347,2.9651,4.9372,1.6001,-0.7575,-1.0393,0.8769,-0.6531,-0.9102,0.8879,-1.3799,-1.8379,0.7751,-0.7359,-1.0153,0.8695,-0.2444,-0.3300,0.9539
50%,20340.0000,1.0200,51.8000,-3.7600,5.9524,4.6900,-4.7200,-8.1100,-5.8100,7.9800,2.3100,-2.2800,-4.2400,-13.5800,16.3600,-20.3300,24.0700,2.9300,4.1134,3.1300,4.3200,2.2595,2.7300,0.8100,70.2900,1.8600,15.8100,2.6400,3.0100,20.3800,13.2750,0.7450,16.6100,23.9880,9957.0310,888.0428,6.2175,13.3910,2.5893,5.2635,10.4202,2.3230,0.6782,1.1076,1.1290,0.1905,0.3104,1.0357,-0.5367,-0.7692,0.9071,-0.1354,-0.1957,0.9756,0.3381,0.4813,1.0643
75%,53030.0000,1.3000,56.2200,7.8700,8.8394,7.4600,0.4150,3.1650,7.8650,26.7400,5.3850,2.0100,5.9950,-7.1250,22.3000,-11.5700,40.1600,5.7750,5.2814,4.0450,5.5000,5.2748,5.6850,1.0350,133.1100,3.3500,22.9925,5.5600,6.3650,32.7325,24.3975,1.5300,25.1675,62.0671,9957.0310,888.0428,9.5964,29.4031,4.6436,7.5977,18.5123,3.5185,1.9980,3.6136,1.4306,1.1906,1.7973,1.2357,0.5086,0.7365,1.0930,0.6187,0.9130,1.1201,0.8627,1.2875,1.1703
max,3130150.0000,5.0600,78.9900,105.6600,45.5194,59.2200,81.3200,52.8700,152.4100,1014.3500,62.3100,75.0500,98.7000,8.4200,136.0100,8.2900,1647.6200,47.5700,11.4072,10.8800,12.2300,217.5547,315.2700,9.2800,1096.8700,15.4900,166.1200,1723.5200,4

In [8]:
print(f'date_str: {date_str}')
df_data.head()

date_str: 2025-04-24


,Company,Info,"MktCap, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Fwd P/E,P/S,P/B,P/FCF,ROE %,Debt/Eq,Oper M %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",3130150.0000,1.2800,51.3800,-16.7900,7.8743,7.2600,-6.8700,-6.8400,-11.8900,25.6500,3.0100,-5.0800,-8.5000,-16.6500,23.1400,-19.8900,25.3700,9.7000,4.6552,2.5400,4.9500,47.0682,61.2700,0.7700,208.3700,1.8400,26.1900,7.9100,46.9500,31.8400,136.5200,1.4500,31.7600,80.6701,9957.0310,888.0428,9.6300,23.0568,3.9049,6.2602,11.1003,2.5127,0.8612,1.5845,1.1613,0.1840,0.2850,1.0358,-0.9594,-1.3857,0.8283,-0.4254,-0.6087,0.9178,0.6840,1.0178,1.1436
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2879180.0000,0.9800,53.8800,-8.1100,7.8470,4.2200,-1.9900,-13.3000,-7.5200,-3.4100,3.0700,-0.0700,-6.8300,-7.6300,12.3300,-17.3100,12.3300,12.9600,3.3462,2.4400,3.5100,22.1123,25.2500,0.8800,387.3000,3.4500,26.0500,11.0000,9.5100,41.1100,34.2900,0.3400,44.9600,44.4284,9957.0310,888.0428,8.2031,17.5381,3.2096,1.2219,1.8995,1.2080,1.3725,2.8900,1.2826,0.5617,1.0440,1.1112,-1.5374,-2.2744,0.7455,-0.7315,-1.0429,0.8716,-0.1273,-0.1783,0.9771
NVDA,NVIDIA Corp,"Technology, Semiconductors",2596890.0000,2.1100,48.2400,-20.7500,9.8235,1.8600,-11.8200,-27.7100,-25.9400,33.8400,1.0400,-7.9700,-15.3100,-25.8000,22.8700,-30.5000,36.0600,6.8700,6.4549,3.5400,6.4400,217.5547,315.2700,0.6900,106.4300,3.6200,18.8900,19.9000,32.8400,42.6700,119.1800,0.1300,62.4200,347.0916,9957.0310,888.0428,5.0026,8.6786,2.0934,-0.2734,-0.3631,0.9587,1.1871,2.2707,1.2373,-0.5525,-0.9010,0.9046,-0.6254,-0.9094,0.8976,-0.6372,-0.8691,0.8943,0.6602,0.9563,1.1190
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",1979630.0000,1.3000,50.4600,-14.9700,11.4870,7.0000,-9.3200,-20.7600,-1.3400,5.2500,2.4600,-5.4800,-6.3900,-20.0900,15.5900,-23.0800,23.0400,9.4800,5.0820,3.4200,5.2500,41.9393,49.4300,0.8500,186.5400,3.2900,24.9900,3.1000,6.9100,60.2100,24.2900,0.5200,10.8700,85.0378,9957.0310,888.0428,9.2623,20.4775,3.5799,1.8775,3.3802,1.3110,1.3845,2.7160,1.2522,-0.3348,-0.5044,0.9443,-2.0411,-2.8035,0.7029,-0.0803,-0.1169,0.9864,0.2643,0.3748,1.0472
GOOGL,Alphabet Inc,"Communication Services, Internet Content & Inf...",1954980.0000,1.0000,51.7200,-15.8600,7.8621,3.8800,-6.6100,-19.5500,-2.9200,1.9200,3.5800,-3.2500,-7.6200,-14.8000,13.3400,-23.0700,13.3400,5.8600,3.6791,2.6800,3.8400,42.6839,35.0800,1.2200,159.2800,2.5300,15.9000,5.5900,5.9800,26.8700,32.9100,0.0900,32.4600,1786.4239,9957.0310,888.0428,8.6365,18.0698,3.2766,3.2829,5.9107,1.5669,1.9599,4.0045,1.3816,-0.2670,-0.4194,0.9573,-2.0078,-2.6746,0.7181,-0.4415,-0.6298,0.9290,0.1135,0.1640,1.0201
